In [13]:
import sympy as sp
from sympy import Matrix as mtx
from sympy import symbols as syms
import math
from collections import deque
import copy
import random

### Monomials
Lets start with monomials

In [2]:
def init_mono(var, degree, coeff):
    if len(coeff) != degree + 1:
        raise Exception("Degree coeff doesn't match")
        return
    poly = coeff[0]
    base = [1]
    for i in range(1, degree+1):
        base.append(var**i)
    return base, coeff

In [3]:
x = syms('x')
b, c = init_mono(x, 4, [2, 3, 5, 7, 1])
b

[1, x, x**2, x**3, x**4]

In [4]:
def init_scaled_mono_basis(var, degree, poly):
    p = 1
    mb, c = poly
    for i in range(1, degree+1):
        p *= math.factorial(i)
    const = math.factorial(degree) / p
    base = [1]
    coeff = [c[0]/p]
    for i in range(1, degree+1):
        base.append(var**i)
        coeff.append(c[i] / p)
    return base, coeff

In [5]:
init_scaled_mono_basis(x, 4, (b, c))

([1, x, x**2, x**3, x**4],
 [0.006944444444444444,
  0.010416666666666666,
  0.017361111111111112,
  0.024305555555555556,
  0.003472222222222222])

# Multivariate Polynomial

### Initalize a polynomial using monomial bases

In [14]:
def init_poly(var_num, degree, coeff=None):
    '''
    Init a multivariate polynomial with given num of var and num of degree.
    The base used here is monomial
    '''
    var_list = deque()
    for i in range(var_num):
        var_list.append(syms('x_' + str(i)))
    terms = []
    for i in range(degree, -1, -1):
        terms += gen_terms(var_list, i, 1)
    if coeff == None:
        coeff = []
        for i in range(len(terms)):
            coeff.append(random.randint(-100, 100))
    return var_list, terms, coeff

In [15]:
def gen_terms(var_list, total_degree, term):
    '''
    var_list is of type deque
    '''
    terms = []
    if len(var_list) == 1:
        term *= var_list[0] ** total_degree
        terms.append(term)
        return terms
    elif total_degree == 0:
        term *= 1
        terms.append(term)
        return terms
    else:
        for i in range(total_degree, -1, -1):
            var_list_further = copy.deepcopy(var_list)
            var = var_list_further.popleft()
            term_further = term * var ** i
            terms += gen_terms(var_list_further, total_degree - i, term_further)
        return terms

In [26]:
v, d, c = init_poly(2, 2, coeff=None)

### Convert to Quadratic form

Given a polynomial (terms, coeffs), and a bases (basis), convert it into the form $b^TQb$, where Q is a matrix. 
We want to generate the linear equations that constraint Q
- Q should be a symmetric matrix

In [27]:
v

deque([x_0, x_1])

In [28]:
d

[x_0**2, x_0*x_1, x_1**2, x_0, x_1, 1]

In [29]:
c

[-39, -33, 80, 87, 95, 70]

In [30]:
basis = [1, v[0], v[1]]

In [86]:
def gen_linear_eq(basis, terms, coeffs):
    Q_dim = len(basis)
    mtx_var = []
    for i in range(Q_dim):
        for j in range(i, Q_dim):
            mtx_var.append(syms('q_(' + str(i) + ')(' + str(j) + ')'))
    A = mtx.zeros(Q_dim)
    counter = 0
    for i in range(Q_dim):
        for j in range(i, Q_dim):
            if i == j:
                A[i, j] = c[counter]
            else:
                A[i, j] = 0.5 * c[counter]
            counter += 1
    return (A, A.condition_number())

In [88]:
A, num = gen_linear_eq(basis, d, c)

In [89]:
A

Matrix([
[-39, -16.5, 40.0],
[  0,    87, 47.5],
[  0,     0,   70]])

In [90]:
num

3.66054824437078